**NOTA**: Si detectas algún error en este Colab, pon un mensaje en el foro para que lo podamos solucionar o envía un correo.

**NOTA**: Las imágenes utilizadas en este colab han sido descargadas de https://pixabay.com/es/

# 1 Detección de objetos

En este colab vamos a trabajar con detección de objetos. Este término, como puedes imaginar, hace referencia al proceso de encontrar instancias de objetos reales en imágenes (o vídeos). Una vez realizado el proceso de **detección**, podríamos realizar un posterior proceso de **clasificación** para asignar a un objeto una categoría específica (p.e., clasificación de caras).

La detección de objetos tiene gran importancia en aplicaciones de procesamiento de imágenes y vídeos, ya que se utiliza en distintas aplicaciones como reconocimiento facial, aplicaciones médicas con imágenes, etc. 

**OpenCV** nos ofrece diversas funcionalidades para detectar objetos. A continuación, veremos algunas de ellas. Para ello, vamos a trabajar con el mismo repositorio de imágenes que utilizamos en el colab anterior. Si no lo tienes, aprovecha para clonarlo:

In [ ]:
git clone https://gitlab.com/robotica_gti/opencv.git

## 1.1 Detección por color

La detección por color nos permite identificar objetos a partir de un color establecido. Vamos a suponer que queremos identificar el jugador con el uniforme azul que aparece en la imagen **futbol_reduced.jpg**. Para ello, podemos seguir los siguientes pasos:

1. Convertir la imagen al espacio de color HSV
2. Identificar los valores máximos y mínimos en HSV del color que queremos detectar
3. Crear una máscara con los valores máximo y mínimo identificados
4. Aplicar la máscar a la imagen original

En primer lugar, no olvides activar el entorno virtual **rosenv**. 

A continuación, el primer paso será leer la imagen y **convertirla** al espacio de color HSV. Para ello, utilizaremos la función **cvtColor**: https://docs.opencv.org/4.5.5/db/d64/tutorial_js_colorspaces.html. Esta función recibe como primer parámetro la imagen cargada con OpenCV (i.e. objeto ndarray) y el tipo de transformación a realizar. En el siguiente enlace tienes las distintas transformaciones que se pueden realizar: https://docs.opencv.org/master/d8/d01/group__imgproc__color__conversions.html.

Puedes aprovechar para mostrar por pantalla el resultado:

In [ ]:
img = cv2.imread('futbol_reduced.jpg')

hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

cv2.imshow('hsv',hsv)
cv2.waitKey(0) #aprieta una tecla 
cv2.destroyAllWindows()

A continuación, debemos **identificar** los valores máximos y mínimos en HSV del color que queremos detectar. Para ello, debemos indicar el color en HSV que queremos. Como estos valores son difíciles de generar, es mejor que usemos una herramienta color picker (como [ColorZilla](https://chrome.google.com/webstore/detail/colorzilla/bhlhnicpbhignbdhedgjhgdocnmhomnp)) para seleccionar el color en RGB. Cuando lo tenemos seleccionado, podemos escribir el siguiente código en un terminal de Python, que nos mostrará cuál es el valor en HSV que debemos indicar. En este ejemplo, lo vamos a hacer con el color azul:

In [ ]:
# esto lo ejecutamos a parte en un terminal de Python para obtener los valores
import numpy as np
import cv2

dark_blue=np.uint8([[[255,0,0]]])
hsv_dark_blue = cv2.cvtColor(dark_blue,cv2.COLOR_BGR2HSV)
print(hsv_dark_blue)
#[[[120 255 255]]]  --> esto es el color en HSV

Cuando ya tengamos definidos los rangos de color que queremos capturar (en nuestro ejemplo, desde azul claro hasta azul oscuro), los especificaremos:

In [ ]:
lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])

El siguiente paso vamos a crear una **máscara** con los valores mínimo y máximo identificados. Básicamente, vamos a crear una imagen con dos colores: blanco y negro. En blanco estarán los píxeles que tengan un color entre estos dos valores indicados (i.e. un color que consideramos como azul) y en negro estarán el resto de píxeles (i.e. un color que no consideramos como azul). Para ello, utilizaremos la función **inRange**, que nos devolverá como decimos, una máscara de los azules que encuentra. Puedes probar a imprimirla para verlo:

In [ ]:
mask = cv2.inRange(hsv, lower_blue, upper_blue)

cv2.imshow('mask',mask)
cv2.waitKey(0) #aprieta una tecla 
cv2.destroyAllWindows()

Finalmente, vamos a aplicar la máscara a la imagen original mediante la función **bitwise_and**. Esta función recibe la imagen y la máscara, y nos devuelve una nueva imagen con aquellos píxeles que corresponden con la máscara. Puedes imprimir el resultado:

In [ ]:
res = cv2.bitwise_and(img, img, mask= mask)

cv2.imshow('res',res)
cv2.waitKey(0) #aprieta una tecla 
cv2.destroyAllWindows()

## 1.2 Ejercicio

5. Una vez hayas realizado la identificación por color, te planteamos que muestres la imagen original con un bounding box donde aparezca el jugador en azul. Para ello, puedes ver en qué píxeles de la máscara hay un valor distinto de negro y dibujar un rectángulo que los englobe a todos.

## 1.2 Detección por contornos

Los contornos son las líneas que permiten trazar los límites de una imagen o de una figura. Básicamente, son trazos (líneas rectas, curvas, ángulos, etc.) del objeto que estamos observando. En este sentido, podemos ver el contorno como el perímetro o borde de un objeto. 

OpenCV nos ofrece también varias funcionalidades para detectar contornos de objetos que se observan en una imagen. Algunas tendrán un mejor resultado que otras, pero el objetivo es probar alguna de ellas. En nuestro caso, vamos a seguir los siguientes pasos:
1. Transformar la imagen a escala de grises.
2. Segmentar la imagen.
3. Buscar los contornos de la imagen.
4. Dibujar los contornos en la imagen original.

El primer paso consiste en cargar la imagen (en nuestro caso **oficina2.jpg**) y  transformarla a **escala de grises**, puesto que su procesamiento es menos pesado, ya que el coste computacional de trabajar con imágenes a color crece de manera exponencial. Ten en cuenta que trabajar a color, por ejemplo en el espacio RGB, implica trabajar con 3 imágenes diferentes (una para cada componente de color). La conversión a escala de grises es algo que ya vimos en el colab anterior, por lo que debería resultarte familiar:

In [ ]:
img = cv2.imread('oficina2.jpg')

img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

A continuación, vamos a **segmentar** la imagen mediante la detección de contornos. En este sentido, se buscarán curvas de puntos con un principio y un final conectados que se identificarán como contornos. Esto representa un conjunto contínuo de píxeles sin huecos ni saltos del mismo color e instensidad. Para ello, aplicaremos un proceso de umbralización (threshold) binaria. Este proceso consiste en comprobar si la intensidad de un píxel es mayor que un umbral y se establece como valor máximo. Por contra, si está por debajo, se establece al valor mínimo. Básicamente, transforma los valores por encima del umbral a 255 y por debajo, a 0:

In [ ]:
ret, umbral = cv2.threshold(img_gray, 155, 255, cv2.THRESH_BINARY)

A continuación, utilizaremos la función **findContours** para buscar los contornos. Esta función necesita los siguientes parámetros:
* La imagen a procesar (la imagen que nos devuelve la función anterior).
* El [modo de recuperación de contornos](https://docs.opencv.org/4.5.5/d3/dc0/group__imgproc__shape.html#ga819779b9857cc2f8601e6526a3a5bc71).
* El [método de aproximación del contorno](https://docs.opencv.org/4.5.5/d3/dc0/group__imgproc__shape.html#ga4303f45752694956374734a03c54d5ff).

Además, esta función devuelve los siguientes valores:
* Una lista con todos los contornos de la imagen, donde cada contorno corresponde con un array Numpy de coordenadas (x,y) que delimitan los puntos frontera del objeto.
* Una jerarquía que contiene información sobre la topología de la imagen.

In [ ]:
contornos, jerarquia = cv2.findContours(umbral, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

Finalmente, vamos a **dibujar** los contornos en la imagen original mediante la función **drawContours**. Esta función recibe la imagen original, la lista con los contornos que queremos dibujar, el índice sobre la lista con el contorno que queremos dibujar (un -1 significa todos), el color con el que se dibujan y el grosor:

In [ ]:
cv2.drawContours(img, contornos, -1, (0,165,255), 3)

cv2.imshow('Logo color', img)
cv2.imshow('Logo gris', img_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

Como curiosidad, comprueba cómo detecta los bordes en la imagen **escritorio.jpg**.

A continuación, tienes dos enlaces que puedes utilizar para ampliar información:
* https://docs.opencv.org/master/d4/d73/tutorial_py_contours_begin.html
* https://docs.opencv.org/4.5.5/df/d0d/tutorial_find_contours.html


## 1.3 Detección de bordes (algoritmo de Canny)

Una funcionalidad similar a la anterior es la detección de bordes mediante el algoritmo de Canny. Este algoritmo puede conseguir una mayor precisión para encontrar contornos y detectar una amplia gama de bordes en imágenes.

En esta ocasión, repetiremos los mismos pasos que para el caso anterior, pero variaremos la fase de segmentación de la imagen, la cual haremos mediante el algoritmo de Canny:

1. Transformar la imagen a escala de grises.
2. **Segmentar la imagen.**
3. Buscar los contornos de la imagen.
4. Dibujar los contornos en la imagen original.

Teniendo en cuenta esto. Vamos a ir directamente al punto donde ya tenemos una imagen en escala de grises. Entonces, utilizaremos la función **GaussianBlur** para eliminar ruido. Esta función requiere de los siguientes parámetros:
* La imagen original a procesar.
* El tamaño del kernel (indica cómo calcular el valor de un píxel combinándolo con los vecinos, y debe ser un valor impar).
* La desviación estándar x,y del kernel Gaussiano.

Como resultado, obtenemos una imagen a la cual le hemos aplicado el filtro (i.e. array Numpy):

In [ ]:
imagen_desenfocada = cv2.GaussianBlur(img,(5,5),0)

Con esta imagen, utilizaremos la función **Canny** para la detección de bordes. Esta función recibe la imagen generada después de aplicar el filtro anterior y dos valores (mínimo y máximo) de umbralización:

In [ ]:
imagen_canny = cv2.Canny(imagen_desenfocada, 50, 50)

Después, podemos utilizar las mismas funciones que antes para mostrar el resultado. En este caso, probamos unos valores diferentes en los parámetros:

In [ ]:
contours, _ = cv2.findContours(imagen_canny.copy(), mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)

cv2.drawContours(image=img,contours=contours,contourIdx=-1,color=(0,255,0),thickness=2, lineType=cv2.LINE_AA)

cv2.imshow("Contornos", img)
cv2.imshow('Imagen desenfocada', imagen_desenfocada)
cv2.imshow('Imagen tras Canny', imagen_canny)

cv2.waitKey(0)
cv2.destroyAllWindows()

Prueba ahora a aplicar este script con la imagen **escritorio.jpg** y observa si existen diferencias. A continuación, tienes un enlace para ampliar información:
* https://docs.opencv.org/master/da/d22/tutorial_py_canny.html

## 1.4 Detección de figuras geométricas

En ocasiones también es interesante detectar figuras geométricas. Para ello, debemos seguir los siguientes pasos:
1. Umbralizar la imagen a procesar.
2. Buscar los contornos.
3. Iterar sobre los contornos encontrados.

Los dos primeros pasos son equivalentes a los utilizados anteriormente. En esta ocasión, lo realizaremos sobre la imagen **figuras.png**:

In [ ]:
image = cv2.imread('figuras.png')

cv2.imshow('Imagen original', image)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Umbralizamos la imagen
_, threshold = cv2.threshold(img_gray, 240, 255, cv2.THRESH_BINARY)

# Buscamos los contornos
contornos, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

Como hemos comentado anteriormente, la función **findContours** devuelve los contornos encontrados. A continuación, recorreremos la lista realizando los siguientes pasos para cada contorno:
1. Identificamos los polígonos con la función **approxPolyDP**. Esta función recibe un contorno y un valor *epsilon* que indica la precisión de la aproximación a realizar. El tercer parámetro indica si se trata de un contorno cerrado o no.
2. Averiguamos el número de curvas del polígono identificado con la función **len**.
3. Dependiendo del número de curvas, ponemos un texto diferente.

A continuación tienes el código de este proceso para identificar figuras:

In [ ]:
# Iteramos sobre los contornos
for contorno in contornos:
 
    poligonoAproximado = cv2.approxPolyDP(contorno, 0.01* cv2.arcLength(contorno, True), True)
    cv2.drawContours(image, [poligonoAproximado], 0, (0,0,0), 5)

    # Buscamos las coordenadas donde queremos escribir el nombre de la forma
    x = poligonoAproximado.ravel()[0]
    y = poligonoAproximado.ravel()[1] - 5

    # Basandonos en el núnmero de curvas poligonales determinamos de qué
    # forma geométrica se trata en cada caso
    numeroCurvas = len(poligonoAproximado)

    if numeroCurvas == 3:
        cv2.putText(img, 'Triangulo', (x,y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0))
    elif numeroCurvas == 4:
        x1, y1, w, h = cv2.boundingRect(poligonoAproximado)
        relacionDeAspecto = float(w)/h
        if (relacionDeAspecto > 0.95 and relacionDeAspecto <= 1.05):
            cv2.putText(img, 'Cuadrado', (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
        else:
            cv2.putText(img, 'Rectangulo', (x,y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0))
    elif numeroCurvas == 5:
        cv2.putText(img, 'Pentagono', (x,y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0))
    elif numeroCurvas == 10:
        cv2.putText(img, 'Estrella', (x,y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0))
    else:
        cv2.putText(img, 'Circulo', (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))

cv2.imshow('Gris', img_gray)
cv2.imshow('Formas geometricas', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


En los siguientes enlaces puedes ampliar información sobre este procedimiento:
* https://docs.opencv.org/4.5.5/dc/dcf/tutorial_js_contour_features.html
* https://docs.opencv.org/master/d3/dc0/group__imgproc__shape.html


## 1.5 Blob

Un **Blob** (Binary Large Object) es un grupo de píxeles conectados en una imagen que comparten alguna propiedad en común (p.e. un color en BGR o escala de grises). Relacionado con un Blob, nos referimos al **centroide** de una figura como la media aritmética de todos los puntos de la misma. En el contexto de la visión artificial, el centroide es el punto medio de una figura o una forma.

OpenCV nos ofrece funcionalidades para detectar el **centro** de un Blob a partir del momento de una imagen. Un **momento** puede definirse como el promedio ponderado particular de las intensidades de píxeles de la imagen. El centroide se puede calcular con la siguiente fórmula:

$$C_x = \frac{M_{10}}{M_{00}}  \hspace{1cm}C_x = \frac{M_{01}}{M_{00}}$$

Donde $M_{00}$ es el área del objeto y $M_{10},M_{01}$ es el centro de masas del objeto.

Para encontrar el centroide de un Blob, deberemos seguir los siguientes pasos:
1. Convertir la imagen a escala de grises.
2. Llevar a cabo una binarización de la imagen mediante el umbral.
3. Encontrar los contornos de la imagen.
4. Encontrar el centro de la imagen tras calcular sus momentos.

Los tres primeros pasos ya los hemos utilizado anteriormente:

In [ ]:
img = cv2.imread('figuras.png')

# convertimos a escala de grises
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# convertimos la imagen a escala de grises en binaria
ret, img_binary = cv2.threshold(img_gray, 127, 255, 0)

# buscamos los contornos en la imagen binaria
contornos, jerarquia = cv2.findContours(img_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

A continuación, vamos a recorrer los contornos que se han detectado. En primer lugar, utilizamos la función **moments** para obtener los momentos. Después, calculamos el centroide siguiendo las fórmulas anteriores. Finalmente, dibujamos el centroide y ponemos también un texto:

In [ ]:
for contorno in contornos:
    # calculamos los momentos para cada contorno
    M = cv2.moments(contorno)
    
    # calculamos las coordenadas x,y del centroide
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
        
    cv2.circle(img, (cX, cY), 5, (0, 0, 255), -1)
    cv2.putText(img, "centroide", (cX - 25, cY - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),2)

# mostramos la imagen
cv2.imshow("Imagen", img)

cv2.waitKey(0)

A continuación tienes dos enlaces que puedes consultar para ampliar información:

* https://learnopencv.com/blob-detection-using-opencv-python-c/
* https://docs.opencv.org/4.5.5/d0/d7a/classcv_1_1SimpleBlobDetector.html